# Trabalho de extração

## Carregamento dos dados e pré-processamento

In [136]:
import pandas as pd

# Leitura de dados
RNA_cpm = pd.read_csv("data_mrna_seq_cpm.txt", index_col=0, header = 0, sep="\t")
meta_patient = pd.read_csv("data_clinical_patient.txt", sep="\t", skiprows=4, index_col=0)
meta_sample = pd.read_csv("data_clinical_sample.txt", sep="\t", skiprows=4, index_col=0)

In [137]:
print(RNA_cpm.info())
print(RNA_cpm.head())

<class 'pandas.core.frame.DataFrame'>
Index: 22843 entries, TSPAN6 to RP11-114I8.4
Columns: 452 entries, Entrez_Gene_Id to aml_ohsu_2018_16-01272
dtypes: float64(452)
memory usage: 78.9+ MB
None
             Entrez_Gene_Id  aml_ohsu_2018_12-00023  aml_ohsu_2018_12-00051  \
Hugo_Symbol                                                                   
TSPAN6                  NaN                0.220678                1.659656   
DPM1                    NaN               29.791548               29.363142   
SCYL3                   NaN               19.336921               18.734961   
C1orf112                NaN                6.868607                2.617150   
FGR                     NaN              801.392633               17.266804   

             aml_ohsu_2018_12-00066  aml_ohsu_2018_12-00150  \
Hugo_Symbol                                                   
TSPAN6                     0.070215                1.709459   
DPM1                      23.311415               32.137834   

Verificámos assim que o dataset é um dataframe com uma dimensão de 22843 entradas nas linhas, correspondentes aos genes, e 452 colunas, correspondentes às amostras. Dado que a coluna 'Entrez_Gene_Id' aparenta estar vazia, a dimensão do dataset é a esperada, apresentando 22843 genes e 451 amostras.
De seguida, procedemos à eliminção da coluna com valores omissos e verificação de duplicados no nome dos genes. 

In [138]:
print('Dimensão original do dataframe:', RNA_cpm.shape)
# Verificar se há valores nulos
if RNA_cpm.isna().sum().sum() != 0:
    print('Há valores nulos no dataframe.')
# Retirar a coluna 'Entrez_Gene_Id'
RNA_cpm = RNA_cpm.drop(columns=['Entrez_Gene_Id'])
print('Dimensão do dataframe após remoção da coluna vazia:', RNA_cpm.shape)
if RNA_cpm.isna().sum().sum() == 0:
    print('Já não há valores nulos no dataframe.')

Dimensão original do dataframe: (22843, 452)
Há valores nulos no dataframe.
Dimensão do dataframe após remoção da coluna vazia: (22843, 451)
Já não há valores nulos no dataframe.


In [139]:
# Verificação de nome de genes duplicados
duplicated_genes = RNA_cpm[RNA_cpm.index.duplicated(keep= "first") ].index
if duplicated_genes.size == 0:
    print('Não há genes duplicados.')
else:
    print(f'Há {len(duplicated_genes)} genes duplicados:\n', duplicated_genes)

Há 6 genes duplicados:
 Index(['BTBD8', 'COMMD9', 'S1PR3', 'MYO18A', 'ENO1', 'MACF1'], dtype='object', name='Hugo_Symbol')


In [140]:
# Rename duplicated genes
def renomear_duplicados(dados):
    index_list = dados.index.tolist()
    for gene_dup in duplicated_genes:
        for pos, gene in enumerate(dados.index):
            if gene == gene_dup:
                index_list[pos] = gene + "_duplicado"
                print(index_list[pos])
                break            
    dados.index = index_list
    if dados[dados.index.duplicated()].index.size == 0:
        return "Não há genes duplicados"
    else:
        return "Ainda há genes duplicados"

In [141]:
# Renomeando duplicados
renomear_duplicados(RNA_cpm)

BTBD8_duplicado
COMMD9_duplicado
S1PR3_duplicado
MYO18A_duplicado
ENO1_duplicado
MACF1_duplicado


'Não há genes duplicados'

In [142]:
if 'BTBD8' in RNA_cpm.index and 'BTBD8_duplicado' in RNA_cpm.index:
    print("BTBD8 estava duplicado, mas o duplicado foi renomeado com sucesso")

BTBD8 estava duplicado, mas o duplicado foi renomeado com sucesso


## Pré-processamento de metadados


In [143]:
print(meta_patient.info())

<class 'pandas.core.frame.DataFrame'>
Index: 562 entries, aml_ohsu_2018_17 to aml_ohsu_2018_4380
Data columns (total 21 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   SEX                                      562 non-null    object 
 1   ETHNICITY                                562 non-null    object 
 2   AGE_AT_DIAGNOSIS                         556 non-null    float64
 3   MDSMPN_DIAGNOSIS_AT_ACQUISITION          562 non-null    bool   
 4   NON_AML_MDSMPN_DIAGNOSIS_AT_ACQUISITION  562 non-null    bool   
 5   PRIOR_CANCER                             74 non-null     object 
 6   CHEMOTHERAPY                             562 non-null    object 
 7   PRIORRADIATIONFOROTHERCANCER             562 non-null    object 
 8   PRIOR_MDS                                562 non-null    object 
 9   MDS_TWO_MONTHS_PRIOR_AML                 562 non-null    object 
 10  PRIOR_MDSMPN             

In [145]:
print(meta_sample.info())

<class 'pandas.core.frame.DataFrame'>
Index: 672 entries, aml_ohsu_2018_163 to aml_ohsu_2018_2747
Data columns (total 74 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   SAMPLE_ID                                         672 non-null    object 
 1   SAMPLE_TIMEPOINT                                  442 non-null    object 
 2   FUSION                                            160 non-null    object 
 3   SPECIFIC_DIAGNOSIS_AT_INCLUSION                   672 non-null    object 
 4   ELN_2017                                          672 non-null    object 
 5   ELN_2008                                          658 non-null    object 
 6   SPECIFIC_DIAGNOSIS_AT_ACQUISITION                 672 non-null    object 
 7   GROUP                                             672 non-null    object 
 8   SAMPLE_SITE                                       672 non-null    object 


In [146]:
print(meta_sample.columns)

Index(['SAMPLE_ID', 'SAMPLE_TIMEPOINT', 'FUSION',
       'SPECIFIC_DIAGNOSIS_AT_INCLUSION', 'ELN_2017', 'ELN_2008',
       'SPECIFIC_DIAGNOSIS_AT_ACQUISITION', 'GROUP', 'SAMPLE_SITE', 'PLATFORM',
       'RNA_SEQUENCED', 'WHOLE_EXOME_SEQUENCING', 'EX_VIVO_DRUG_TESTING',
       'RNA_SEQ_ANALYSIS', 'EXOME_SEQ_ANALYSIS', 'DRUG_TESTING_IN_ANALYSIS',
       'CUMULATIVE_TREATMENT_TYPE_COUNT', 'CUMULATIVE_TREATMENT_TYPES',
       'CUMULATIVE_TREATMENT_REGIMEN_COUNT', 'CUMULATIVE_TREATMENT_REGIMENS',
       'CUMULATIVE_TREATMENT_STAGE_COUNT', 'CUMULATIVE_TREATMENT_STAGES',
       'INDUCTION_RESPONSE', 'TYPE_INDUCTION_TREATMENT',
       'DURATION_OF_INDUCTION_TREATMENT', 'MOST_RECENT_TREATMENT_TYPE',
       'CURRENT_REGIMEN', 'CURRENT_STAGE', 'MOST_RECENT_TREATMENT_DURATION',
       'KARYOTYPE_SAMPLE_ACQUISITION_INTERVAL_DIFFERENCE',
       'PB_BASOPHILS_PERCENTAGE', 'BM_BLAST_PERCENTAGE', 'PB_BLAST_PERCENTAGE',
       'CEBPA_BIALLELIC_MUTATION', 'PB_EOSINOPHILS_PERCENTAGE',
       'PB_IMMATURE_

Primeiramente, verificaram-se informações sobre os dataframes referentes a metadados de pacientes e de amostras carregados. 

Nos metadados dos pacientes estão descritas 21 variáveis correspondentes às 21 colunas do dataframe, sendo que nas linhas estão os 562 pacientes cada um representado através de um ID único. 

Nos metadados das amostras estão descritas 74 variáveis correspondentes às 74 colunas do dataframe, sendo que nas linhas estão as 672 amostras representadas pelo seu ID único. Esta quantidade de amostras superior à dos pacientes é esperada dado que alguns pacientes fornecerão mais do que uma amostra.

É possível notar que nem todas as amostras ou pacientes têm informação para todas as variáveis, existindo alguns valores omissos nos dois dataframes relativos aos metadados.

Para facilitar a análise posterior, um único dataframe foi criado com apenas os metadados referentes a pacientes ou amostras que estejam presentes nos dados de RNA-seq e só com as variáveis que serão usadas posterioremente.


In [157]:
samples_with_rna = meta_sample[meta_sample['RNA_SEQ_ANALYSIS'] == 'Yes']
print(samples_with_rna.info())

<class 'pandas.core.frame.DataFrame'>
Index: 451 entries, aml_ohsu_2018_550 to aml_ohsu_2018_4310
Data columns (total 74 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   SAMPLE_ID                                         451 non-null    object 
 1   SAMPLE_TIMEPOINT                                  317 non-null    object 
 2   FUSION                                            126 non-null    object 
 3   SPECIFIC_DIAGNOSIS_AT_INCLUSION                   451 non-null    object 
 4   ELN_2017                                          451 non-null    object 
 5   ELN_2008                                          445 non-null    object 
 6   SPECIFIC_DIAGNOSIS_AT_ACQUISITION                 451 non-null    object 
 7   GROUP                                             451 non-null    object 
 8   SAMPLE_SITE                                       451 non-null    object 


In [190]:
#Filtrar para ter apenas metadados de pacientes com amostras cujo RNA foi sequenciado
patients_with_rna = meta_patient[meta_patient.index.isin(samples_with_rna.index)]
print(patients_with_rna.info())

<class 'pandas.core.frame.DataFrame'>
Index: 411 entries, aml_ohsu_2018_29 to aml_ohsu_2018_4317
Data columns (total 21 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   SEX                                      411 non-null    object 
 1   ETHNICITY                                411 non-null    object 
 2   AGE_AT_DIAGNOSIS                         408 non-null    float64
 3   MDSMPN_DIAGNOSIS_AT_ACQUISITION          411 non-null    bool   
 4   NON_AML_MDSMPN_DIAGNOSIS_AT_ACQUISITION  411 non-null    bool   
 5   PRIOR_CANCER                             50 non-null     object 
 6   CHEMOTHERAPY                             411 non-null    object 
 7   PRIORRADIATIONFOROTHERCANCER             411 non-null    object 
 8   PRIOR_MDS                                411 non-null    object 
 9   MDS_TWO_MONTHS_PRIOR_AML                 411 non-null    object 
 10  PRIOR_MDSMPN             

In [197]:
# Merge dos metadados
metadados = pd.merge(samples_with_rna, patients_with_rna, on='PATIENT_ID', how='outer')

# Filtragem das variáveis de interesse
variaveis_interesse = ['SAMPLE_ID', 'SAMPLE_TIMEPOINT', 'FUSION', 'ELN_2017', 'ELN_2008',
                        'GROUP', 'SAMPLE_SITE', 'SEX', 'ETHNICITY', 'PRIOR_CANCER',
                        'AGE_AT_DIAGNOSIS',  'AGE_AT_PROCUREMENT',  'DIAGNOSIS_AT_INCLUSION', 'DIAGNOSIS',
                        'CUMULATIVE_TREATMENT_TYPE_COUNT', 'CUMULATIVE_TREATMENT_TYPES', 
                        'CUMULATIVE_TREATMENT_REGIMEN_COUNT', 'CUMULATIVE_TREATMENT_REGIMENS', 
                        'CUMULATIVE_TREATMENT_STAGE_COUNT', 'CUMULATIVE_TREATMENT_STAGES', 
                        'INDUCTION_RESPONSE', 'TYPE_INDUCTION_TREATMENT', 'CANCER_TYPE_DETAILED',
                        'DURATION_OF_INDUCTION_TREATMENT', 'MOST_RECENT_TREATMENT_TYPE', 'CURRENT_REGIMEN', 
                        'CURRENT_STAGE', 'CYTOGENETICS_OTHER', 'SOMATIC_STATUS', 'ONCOTREE_CODE', 'CANCER_TYPE']
print('Nº de colunas do dataframe "metadados":', len(variaveis_interesse))
metadados = metadados[variaveis_interesse]

if metadados.shape == (451, 31):
    print('O dataframe "metadados" tem o número esperado de linhas e colunas, tendo 451 amostras e 31 variáveis (sendo uma das colunas é o ID de cada amostra).')

Nº de colunas do dataframe "metadados": 31
O dataframe "metadados" tem o número esperado de linhas e colunas, tendo 451 amostras e 31 variáveis (sendo uma das colunas é o ID de cada amostra).
